# Retrieval-Augmented Shopping Assistant - EDA

This notebook explores the ABO dataset for initial insights.

In [ ]:
import pandas as pd
import json
import zipfile

In [ ]:
# Path to your zip file
zip_path = '/content/sample_data/listings.zip'

# Open the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # List all files inside the zip
    zip_ref.extractall('/content/sample_data/')

In [ ]:
# Preview
import json
import pandas as pd
def load_json_from_folder(file_name):
  valid_data=[]

  # Load your JSON (string or from a file)
  with open(file_name, 'r', encoding='utf-8') as f:
    for line in f:
      try:
        data = json.loads(line)
        valid_data.append(data)
      except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
  return valid_data

In [ ]:
import os
json_file_path = '/content/sample_data/listings/metadata'
extracted_data=[]
for filename in os.listdir(json_file_path):
  if filename.endswith('.json'):
    file_path = os.path.join(json_file_path, filename)
    valid_data=load_json_from_folder(file_path)
    extracted_data.append(valid_data)

In [ ]:
def remove_language_tag(obj):
    """Recursively remove 'language_tag' keys from nested dicts and lists."""
    if isinstance(obj, dict):
        return {k: remove_language_tag(v) for k, v in obj.items() if k != "language_tag"}
    elif isinstance(obj, list):
        return [remove_language_tag(item) for item in obj]
    else:
        return obj

In [ ]:
cleaned_data = []
# with open("/content/sample_data/listings_1.json", "r", encoding="utf-8") as f:
for line in extracted_data:
    # data = json.loads(line)
    cleaned = remove_language_tag(line)
    cleaned_data.append(cleaned)
cleaned_data[0]

In [ ]:
def flatten_record(record):
    flat = {}
    for key, value in record.items():
        if isinstance(value, list) and all(isinstance(v, dict) and "value" in v for v in value):
            # Join multiple values with comma
            flat[key] = ", ".join(str(v["value"]) for v in value)
        else:
            flat[key] = value
    return flat

In [ ]:
flattened = [flatten_record(rec) for rec in cleaned_data if isinstance(rec, dict)]

# Convert to DataFrame
df = pd.DataFrame(flattened)
df.to_csv('sample_data/shopAssist_Ori.csv', index=False)
df.iloc[:, 25:]

In [ ]:
df.drop(columns=['item_id','model_number','main_image_id','other_image_id', 'domain_name', 'model_year','spin_id','3dmodel_id','finish_type','pattern'],inplace=True)

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
from re import S
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

#store df into csv file
df.to_csv('sample_data/shopAssist.csv', index=False)
#read file from sample_data
df_file = pd.read_csv('sample_data/shopAssist.csv')
#combined all the columns into one
df['combined'] = df_file.apply(lambda x: ' '.join(x.astype(str)), axis=1)
print (df['combined'][0])

In [ ]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [ ]:
def search(query, top_k=2):
    query_vector = model.encode([query])
    distances, indices = index.search(query_vector, top_k)
    df = pd.read_csv('sample_data/shopAssist_Ori.csv')
    return df.iloc[indices[0]]

In [ ]:
result_df = search("Sterling Silver")
images_df = pd.read_csv('sample_data/images.csv')
image_id = result_df['main_image_id'].tolist()
# image_mapping = images_df.set_index('image_id')['path'].to_dict()
image_mapping = images_df[images_df['image_id'].isin(image_id)]['path'].tolist()
for i, (idx,row) in enumerate(result_df.iterrows()):
    result_df['main_image_id'] = "sample_data/"+ image_mapping[i]
result_df
#Map each image_id to its path and prepend "sample_data/"
# result_df['main_image_id'] = result_df['main_image_id'].map(lambda x: f"sample_data/{image_mapping.get(x)}")
# result_df['main_image_id']

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
for url in image_mapping:
  print(url)
  image_path = "sample_data/"+url
  img = mpimg.imread(image_path)
  plt.imshow(img)
  plt.axis('off')
  plt.show()

In [ ]:
result_df.to_json(orient='records').replace("\'", "'")